<a href="https://colab.research.google.com/github/Huangjian2013/ai-demo/blob/main/rag/18-GraphRAG-Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index llama-index-graph-stores-neo4j --quiet


In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from google.colab import userdata
from llama_index.core import SimpleDirectoryReader, Document

In [ ]:

graph_store = Neo4jPropertyGraphStore(
    username=userdata.get("NEO_USER_NAME"),
    password=userdata.get("NEO_PASSWORD"),
    url=userdata.get("NEO_URL")
)
